In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# New Section

In [ ]:
import json
with open(file='/kaggle/input/data_v2/data_v2.json', encoding='utf-8') as file:
    data=json.load(file)


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer


tokenizer = Tokenizer(num_words = 1000)   #maximum word limit 1000
tokenizer.fit_on_texts(data[6]['content']) # fit sentence


In [ ]:
tokenizer.get_config()

In [ ]:
from numpy.lib.arraysetops import unique

category_list = []
filterData = []
garbageCategory = []
garbageContent = []
for i in range(len(data)):
    category = data[i]['category']
    category_list.append(category);
    if category == '-1' or category == '22221' or category == 'we-are' :
      garbageCategory.append(data[i])
      continue
    if data[i]['content'] == '-1' :
      garbageContent.append(data[i])
      continue
    filterData.append(data[i])

print(len(garbageCategory))
print(len(garbageCategory))
len(filterData)

In [ ]:

new_category_list = []

for i in range(len(filterData)):
    new_category_list.append(filterData[i]['category'])


unique_category_list = np.unique(np.array(new_category_list))
print(len(unique_category_list))
print(unique_category_list)


In [ ]:
from collections import Counter
Counter(new_category_list)

In [ ]:

category_count = []
for each_category in unique_category_list:
    count_of_each_category = category_list.count(each_category)
    category_count.append((each_category,count_of_each_category))

category_count.sort();
category_count


In [ ]:
selected_category_list = []

for i in category_count:
     if( i[1] > 2500 ):
         selected_category_list.append(i[0])

print(len(selected_category_list))
selected_category_list

In [ ]:

features = []
target = []


for i in filterData:
    if i['category'] in selected_category_list:
        if target.count(i['category']) < 8000:
          features.append(i['content'])
          target.append(i['category'])


print(len(features))
print(len(target))

# Check dataset
print("Category: --",target[-1])
print("Content: --",features[-1])


In [ ]:
# !pip install bnlp_toolkit
# !pip install git+https://github.com/banglakit/lemmatizer.git#egg=banglakit-lemmatizer

# from bnlp import NLTKTokenizer
# from banglakit import lemmatizer as lem
# from banglakit.lemmatizer import BengaliLemmatizer

# import nltk
# from nltk.stem import WordNetLemmatizer

# nltk_tokenizer = NLTKTokenizer()
# lemmatizer = BengaliLemmatizer()

# # text = "আমার সোনার বাংলা, আমি তোমায় ভালবাসি।"
# # tokens = nltk_tokenizer.word_tokenize(text)
# # lemmatized_text = [lemmatizer.lemmatize(token) for token in tokens]


# # Tokenize the list of text.
# tokens_list = [nltk_tokenizer.word_tokenize(content) for content in features]

# # Apply lemmatization to each token in each list.
# lemmatized_tokens_list = []
# for tokens in tokens_list:
#     lemmatized_tokens = []
#     for token in tokens:
#         lemmatized_tokens.append(lemmatizer.lemmatize(token))
#     lemmatized_tokens_list.append(lemmatized_tokens)

# # Join the tokens back into a single string for each text in the list.
# # lemmatized_text_list = [' '.join(tokens) for tokens in lemmatized_tokens_list]



In [ ]:

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

en_target=le.fit_transform(target)

set(en_target)


In [ ]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(features, en_target, 
                                                    train_size=0.8,
                                                    test_size=0.2,
                                                    random_state=100)


print(len(X_train))
print(len(X_test))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder

vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

# Label encode the class labels
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)


In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.metrics import precision_recall_fscore_support

# Create and fit the model with different alpha values
alphas = [0.1, 0.5, 1.0, 1.5]  # Add more alpha values if needed

for alpha in alphas:
    nb = MultinomialNB(alpha=alpha)
    nb.fit(X_train, y_train)
    predictions = nb.predict(X_test)

    # Evaluate and print the results
    print(f"Alpha: {alpha}")
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, predictions))
    
    print("\nClassification Report:")
    print(classification_report(y_test, predictions))
    
    # Extract precision, recall, and F1-score separately
    precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, predictions, average='weighted')
    
    # Calculate accuracy
    acc = accuracy_score(y_test, predictions)
    
    print("\nMetrics:")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1_score:.4f}")
    print(f"Accuracy: {acc:.4f}")
    print("\n------------------------\n")


In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.metrics import precision_recall_fscore_support

# Assuming X_train, X_test, y_train, y_test are defined

# Train XGBoost model
xgb = XGBClassifier(objective='binary:logistic', learning_rate=0.5, max_depth=20, n_estimators=500)
xgb.fit(X_train, y_train)
xgb_pred = xgb.predict(X_test)

# Evaluate and print the results
print("XGBoost Model Metrics:")
print("Confusion Matrix:")
print(confusion_matrix(y_test, xgb_pred))

print("\nClassification Report:")
print(classification_report(y_test, xgb_pred))



In [ ]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

# Assuming y_test and xgb_pred are defined

# Extract precision, recall, and F1-score separately
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, xgb_pred, average='weighted')

# Calculate accuracy
acc = accuracy_score(y_test, xgb_pred)

print("\nMetrics:")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_score:.4f}")
print(f"Accuracy: {acc:.4f}")
print("\n------------------------\n")



In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.metrics import precision_recall_fscore_support

# Assuming X_train, X_test, y_train, y_test are defined

# Train RandomForest model
rf = RandomForestClassifier(n_estimators=450, criterion='entropy')
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)

# Evaluate and print the results
print("RandomForest Model Metrics:")
print("Confusion Matrix:")
print(confusion_matrix(y_test, rf_pred))

print("\nClassification Report:")
print(classification_report(y_test, rf_pred))

# Extract precision, recall, and F1-score separately
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, rf_pred, average='weighted')

# Calculate accuracy
acc = accuracy_score(y_test, rf_pred)

print("\nMetrics:")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_score:.4f}")
print(f"Accuracy: {acc:.4f}")
print("\n------------------------\n")


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.metrics import precision_recall_fscore_support

# Assuming X_train, X_test, y_train, y_test are defined

# Train Logistic Regression model
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)

# Evaluate and print the results
print("Logistic Regression Model Metrics:")
print("Confusion Matrix:")
print(confusion_matrix(y_test, lr_pred))

print("\nClassification Report:")
print(classification_report(y_test, lr_pred))

# Extract precision, recall, and F1-score separately
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, lr_pred, average='weighted')

# Calculate accuracy
acc = accuracy_score(y_test, lr_pred)

print("\nMetrics:")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_score:.4f}")
print(f"Accuracy: {acc:.4f}")
print("\n------------------------\n")


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.metrics import precision_recall_fscore_support

# A
# Train SVM model
svm = SVC()
svm.fit(X_train, y_train)
svm_pred = svm.predict(X_test)

# Evaluate SVM model
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, svm_pred, average='weighted')
svm_acc = accuracy_score(y_test, svm_pred)

print("SVM Model Metrics:")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_score:.4f}")
print(f"Accuracy: {svm_acc:.4f}")
print("\n------------------------\n")

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming X_train, X_test, y_train, y_test are defined

# Train Gradient Boosting Classifier model
gb = GradientBoostingClassifier()
gb.fit(X_train, y_train)

# Accuracy score, confusion matrix, and classification report of Gradient Boosting Classifier
gb_acc = accuracy_score(y_test, gb.predict(X_test))

print(f"Training Accuracy of Gradient Boosting Classifier is {accuracy_score(y_train, gb.predict(X_train))}")
print(f"Test Accuracy of Gradient Boosting Classifier is {gb_acc} \n")

# Confusion matrix
cm = confusion_matrix(y_test, gb.predict(X_test))
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix - Gradient Boosting Classifier')
plt.show()

# Classification report
classification_rep = classification_report(y_test, gb.predict(X_test))
print(f"Classification Report :- \n{classification_rep}\n")

# Extract precision, recall, and F1-score separately
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, gb.predict(X_test), average='weighted')

# Print precision, recall, and F1-score
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_score:.4f}")

# Compare accuracies and print the best model
best_model = max([('Gradient Boosting Classifier', gb_acc)], key=lambda x: x[1])
print(f"The best model is {best_model[0]} with an accuracy of {best_model[1]:.4f}")


In [ ]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train, y_train)
MultinomialNB(alpha=0.5, class_prior=None, fit_prior=True)
predictions = nb.predict(X_test)
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

In [ ]:
from sklearn.model_selection import cross_val_score

# Perform cross-validation
cv_scores = cross_val_score(xgb, X_train, y_train, cv=5, scoring='accuracy')

# Print cross-validation scores
print("Cross-Validation Scores:", cv_scores)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Assuming X_train, X_test, y_train, y_test are defined

# Train XGBoost model
xgb = XGBClassifier(objective='binary:logistic', learning_rate=0.5, max_depth=20, n_estimators=500)
xgb.fit(X_train, y_train)
xgb_pred = xgb.predict(X_test)

# Train RandomForest model
rf = RandomForestClassifier(n_estimators=450, criterion='entropy')
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)

# Combine predictions using weighted average
combined_pred = 0.7 * xgb_pred + 0.3 * rf_pred  # You can experiment with different weights

# Convert combined predictions to binary
combined_pred_binary = [1 if pred >= 0.5 else 0 for pred in combined_pred]

# Evaluate the combined model
combined_acc = accuracy_score(y_test, combined_pred_binary)

print(f"XGBoost Accuracy: {accuracy_score(y_test, xgb_pred)}")
print(f"RandomForest Accuracy: {accuracy_score(y_test, rf_pred)}")
print(f"Combined Model Accuracy: {combined_acc}")


In [ ]:
!pip install xgboost


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#from pandas_profiling import ProfileReport

%matplotlib inline
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
#from pandas_profiling import ProfileReport
import math

#from statsmodels.stats.outliers_influence import variance_inflation_factor
#from statsmodels.tools.tools import add_constant  
#from statsmodels.stats.outliers_influence import variance_inflation_factor
#from statsmodels.tools.tools import add_constant  
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.datasets import make_classification
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.model_selection import train_test_split
#from imblearn.over_sampling import SMOTE

#from sklearn.metrics import confusion_matrix,plot_roc_curve, classification_report
from sklearn.metrics import mean_absolute_error , mean_absolute_percentage_error , mean_squared_error , accuracy_score
from mlxtend.plotting import plot_confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.impute import KNNImputer
from xgboost import XGBClassifier
#from catboost import CatBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [ ]:
from sklearn.ensemble import RandomForestClassifier
tr= RandomForestClassifier(n_estimators= 450, criterion="entropy")  
tr.fit(X_train, y_train)

In [ ]:

y_pred= tr.predict(X_test)  
from sklearn.metrics import confusion_matrix  
cm= confusion_matrix(y_test, y_pred)  


In [ ]:
!pip install seaborn


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure()
#cf_matrix = confusion_matrix(y_test, model.predict(X_test))
#plt.title('Confusion Matrix: {}'.format(name))
colormap = sns.color_palette("Blues",30)

sns.heatmap(cm, annot = True, fmt = 'g', cmap = 'crest')
plt.show()
    


In [ ]:
!pip install scikit-plot


In [ ]:
import scikitplot as skplt

skplt.metrics.plot_roc(y_test, tr.predict_proba(X_test))
#plt.title('ROC Curves: {}'.format(name))
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_1 = accuracy_score(y_test,y_pred) * 100
accuracy_1

In [ ]:
from sklearn.preprocessing import StandardScaler

# Scaling features between -1 and 1
scaler = StandardScaler(with_mean=False)  # Pass with_mean=False to avoid centering sparse matrices
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)


In [ ]:
!pip install mlxtend


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Models to be used for ML
models = [('Logistic Regression', LogisticRegression()),
          ('Decision Tree Classifier', DecisionTreeClassifier()),
          ('Random Forest', RandomForestClassifier()),
          #('Linear Discriminant Analyzer', LinearDiscriminantAnalysis()),
          ('Ada Boost', AdaBoostClassifier()),
          ('KNN', KNeighborsClassifier()),
          
          ]

models_score = []
for name, model in models:
    model = model #Model Object create
    model.fit(X_train, y_train)
    model.predict(X_test)
    models_score.append([name, accuracy_score(y_test, model.predict(X_test))])
    
    print("Model: ",name)
    print('Validation Accuracy: ', accuracy_score(y_test, model.predict(X_test)))
    print('Training Accuracy: ', accuracy_score(y_train, model.predict(X_train)))
    
    plt.figure()
    cf_matrix = confusion_matrix(y_test, model.predict(X_test))
    #plt.title('Confusion Matrix: {}'.format(name))
    colormap = sns.color_palette("Blues",30)

    sns.heatmap(cf_matrix, annot = True, fmt = 'g', cmap = 'crest')
    plt.show()
    
    import scikitplot as skplt

    skplt.metrics.plot_roc(y_test, model.predict_proba(X_test))
    plt.title('ROC Curves: {}'.format(name))
    plt.show()